In [1]:
import pandas as pd
from datasets import Dataset

/home/user/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df_bench = pd.read_pickle("Datasets/ProCon-bench/StanceBenchmarkProCon_allTrainingDatasets_nopreprocessing.pickle")

dev_df_bench = pd.read_pickle("Datasets/ProCon-bench/StanceBenchmarkProCon_allDevDatasets_nopreprocessing.pickle")

test_df_bench = pd.read_pickle("Datasets/ProCon-bench/StanceBenchmarkProCon_allTestDatasets_nopreprocessing.pickle")

In [3]:
df_arc_train_data = train_df_bench["arc_train"]
df_ibmcs_train_data = train_df_bench["ibmcs_train"]
df_perspectrum_train_data = train_df_bench["perspectrum_train"]
df_fnc1_train_data = train_df_bench["fnc1_train"]
df_iac1_train_data = train_df_bench["iac1_train"]
df_semeval2016t6_train_data = train_df_bench["semeval2016t6_train"]
df_semeval2019t7_train_data = train_df_bench["semeval2019t7_train"]
df_snopes_train_data = train_df_bench["snopes_train"]
df_argmin_train_data = train_df_bench["argmin_train"]
df_scd_train_data = train_df_bench["scd_train"]

In [4]:
df_arc_dev_data = dev_df_bench["arc_dev"]
df_ibmcs_dev_data = dev_df_bench["ibmcs_dev"]
df_perspectrum_dev_data = dev_df_bench["perspectrum_dev"]
df_fnc1_dev_data = dev_df_bench["fnc1_dev"]
df_iac1_dev_data = dev_df_bench["iac1_dev"]
df_semeval2016t6_dev_data = dev_df_bench["semeval2016t6_dev"]
df_semeval2019t7_dev_data = dev_df_bench["semeval2019t7_dev"]
df_snopes_dev_data = dev_df_bench["snopes_dev"]
df_argmin_dev_data = dev_df_bench["argmin_dev"]
df_scd_dev_data = dev_df_bench["scd_dev"]

In [5]:
df_arc_train_data["dataset"] = "arc_train"
train_df_bench["arc_train"] =  df_arc_train_data

df_ibmcs_train_data["dataset"] = "ibmcs_train"
train_df_bench["ibmcs_train"] =  df_ibmcs_train_data

df_perspectrum_train_data["dataset"] = "perspectrum_train"
train_df_bench["perspectrum_train"] = df_perspectrum_train_data

df_fnc1_train_data["dataset"] = "fnc1_train"
train_df_bench["fnc1_train"] = df_fnc1_train_data

df_iac1_train_data["dataset"] = "iac1_train"
train_df_bench["iac1_train"] = df_iac1_train_data

df_semeval2016t6_train_data["dataset"] = "semeval2016t6_train"
train_df_bench["semeval2016t6_train"] = df_semeval2016t6_train_data

df_semeval2019t7_train_data["dataset"] = "semeval2019t7_train"
train_df_bench["semeval2019t7_train"] = df_semeval2019t7_train_data

df_snopes_train_data["dataset"] = "snopes_train"
train_df_bench["snopes_train"] = df_snopes_train_data


df_argmin_train_data["dataset"] = "argmin_train"
train_df_bench["argmin_train"] = df_argmin_train_data

df_scd_train_data["dataset"] = "scd_train"
train_df_bench["scd_train"] = df_scd_train_data


train_df = pd.concat(train_df_bench.values())
train_df = train_df[["dataset", "topic", "text", "label"]]

In [6]:
list_datasets_train = [df_arc_train_data, df_perspectrum_train_data, df_fnc1_train_data]
list_datasets_train2 = [df_iac1_train_data, df_semeval2016t6_train_data, df_semeval2019t7_train_data]
list_datasets_train3 = [df_snopes_train_data, df_argmin_train_data, df_scd_train_data, df_ibmcs_train_data]

In [7]:
list_datasets_dev = [df_arc_dev_data, df_perspectrum_dev_data, df_fnc1_dev_data]
list_datasets_dev2 = [df_iac1_dev_data, df_semeval2016t6_dev_data, df_semeval2019t7_dev_data]
list_datasets_dev3 =  [df_snopes_dev_data, df_argmin_dev_data, df_scd_dev_data, df_ibmcs_dev_data]


In [8]:
list_datasets_str = ['df_snopes_train_data', 'df_argmin_train_data', 'df_scd_train_data', 'df_ibmcs_train_data']

In [9]:
list_datasets_train_str = ['df_arc_train_data', 'df_perspectrum_train_data', 'df_fnc1_train_data']
list_datasets_train2_str = ['df_iac1_train_data', 'df_semeval2016t6_train_data', 'df_semeval2019t7_train_data']
list_datasets_train3_str = ['df_snopes_train_data', 'df_argmin_train_data', 'df_scd_train_data', 'df_ibmcs_train_data']


In [10]:
model_id = "sentence-transformers/nli-roberta-large"

def make_model(params=None):
    return SetFitModel.from_pretrained(
        model_id
    )

In [11]:
from sentence_transformers.losses import CosineSimilarityLoss, ContrastiveLoss, BatchAllTripletLoss

def hyperparameter_search_function(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate",1e-5, 1e-2),
        "loss_class": trial.suggest_categorical("loss_class", [CosineSimilarityLoss, ContrastiveLoss, BatchAllTripletLoss]),
        "num_epochs": trial.suggest_int("num_epochs", 2, 20),
    }

In [12]:
import gc
import torch
gc.collect()

torch.cuda.empty_cache()

In [76]:
#CUDA_LAUNCH_BLOCKING=1

In [17]:
### All the datasets:
import gc
import torch

from collections import defaultdict

import setfit
from setfit import SetFitModel, SetFitTrainer

settings = defaultdict(str)

# trainer = SetFitTrainer(
#         model_init=make_model,
#         train_dataset=train,
#         eval_dataset=dev,
#         num_epochs=1,
#         batch_size=2,
#         num_iterations=5,
#         column_mapping={"text": "text", "label": "label"},
#         )

n = 0

for data in zip(list_datasets_train3, list_datasets_dev3):
    gc.collect()
    torch.cuda.empty_cache()
    
    train = Dataset.from_pandas(data[0], split="train")
    dev = Dataset.from_pandas(data[1], split="dev")
    num_classes = len(train.unique("label"))
    
    trainer = SetFitTrainer(
        model_init=make_model,
        train_dataset=train,
        eval_dataset=dev,
        num_epochs=1,
        batch_size=2,
        num_iterations=5,
        column_mapping={"text": "text", "label": "label"},
        )

    
    best = trainer.hyperparameter_search(hyperparameter_search_function, n_trials=10)
    best
    
    settings[data_name] = best
    #trainer.train(body_learning_rate=1e-5, num_epochs=1)
    
    torch.cuda.empty_cache()
    gc.collect()
    


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2023-06-05 15:04:28,144] A new study created in memory with name: no-name-2528b9f0-d7f5-4a65-a6a3-8b56bd39d603
/home/user/.local/lib/python3.8/site-packages/optuna/distributions.py:535: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'sentence_transformers.losses.CosineSimilarityLoss.CosineSimilarityLoss'> which is of type type.
  warnings.warn(message)
/home/user/.local/lib/python3.8/site-packages/optuna/distributions.py:535: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'sentence_transformers.losses.ContrastiveLoss.ContrastiveLoss'> which is of type type.
  warnings.warn(message)
/home/user/.local/l

AttributeError: 'torch.device' object has no attribute '_apply'

In [ ]:
trainer.apply_hyperparameters(best.hyperparameters, final_model=True) # replaces model_init with a fixed model
trainer.train()

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

In [ ]:
    #trainer.train(learning_rate=1e-2, num_epochs=50)
    trainer.apply_hyperparameters(best.hyperparameters, final_model=True) # replaces model_init with a fixed model
    trainer.train()
    
    result = trainer_set.evaluate()
    best.objective, result
    
    results_f1[data_name] = result
    
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
best = trainer.hyperparameter_search(hyperparameter_search_function, n_trials=10)
best

In [ ]:
trainer.apply_hyperparameters(best.hyperparameters, final_model=True) # replaces model_init with a fixed model
trainer.train()

In [ ]:
metrics = trainer.evaluate()
best.objective, metrics